https://github.com/microsoft/LoRA/blob/main/loralib/ (ref)

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from typing import Optional, List

In [15]:
# todo: implement custom dropout

class LoRALayer:
    def __init__(self, r, lora_alpha, lora_dropout, merge_weights):
        self.r = r
        self.lora_alpha = lora_alpha
        
        if lora_dropout > .0:
            self.lora_dropout = nn.Dropout(p=lora_dropout)
        else:
            self.lora_dropout = lambda x: x
        
        self.merged = False
        self.merge_weights = merge_weights

In [69]:
class Embedding(nn.Embedding, LoRALayer):
    def __init__(self, num_embeddings, embedding_dim, r, lora_alpha, merge_weights, **kwargs):
        nn.Embedding.__init__(self, num_embeddings=num_embeddings, embedding_dim=embedding_dim, **kwargs)
        LoRALayer.__init__(self, r=r, lora_alpha=lora_alpha, lora_dropout=0, merge_weights=merge_weights)
        
        if r > 0:
            self.lora_A = nn.Parameter(self.weight.new_zeros((r, num_embeddings)))
            self.lora_B = nn.Parameter(self.weight.new_zeros((embedding_dim, r)))
            self.scaling = self.lora_alpha/self.r
            
            self.weight.requires_grad = False
        
        self.reset_parameters()
    
    
    def reset_parameters(self):
        nn.Embedding.reset_parameters(self)
        
        if hasattr('lora_A'):
            # there is a mistake in original source code repo
            # raised a pull request to the original repo
            nn.init.normal_(self.lora_A)
            nn.init.zeros_(self.lora_B)
        
    
            
        
        